<a href="https://colab.research.google.com/github/FreNzQuiN/GenerativeAI/blob/main/LLM/runGemma-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
$ pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)

In [ ]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You only allowed to response the result concisely."}]
    },
    {
        "role": "user",
        "content": [
            # {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "Modify this stable diffusion prompt with only change the place/background/settings. Give me only the prompt result: 1girl, goldenglow \\(arknights\\), arknights \\(series\\), dutch angle, looking at viewer, cgdct, cute, solo, smile, light particles, casual, beautiful background, blush, outdoors, park, medium breast, from front, depth of field, very smooth line, year 2024, masterpiece, high score, great score, absurdres"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=200)
print(output[0][0]["generated_text"][-1]["content"])
# Okay, let's take a look!
# Based on the image, the animal on the candy is a **turtle**.
# You can see the shell shape and the head and legs.
